In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import os
import numpy as np
from shapely.geometry import box
from tqdm.notebook import tqdm

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [3]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

In [4]:
%%time
df_polygon = gpd.GeoDataFrame.from_file('./DATA/김해 건축물 정보/김해건물_5179.shp', encoding='cp949')
df_polygon.crs = epsg5179

CPU times: total: 15.4 s
Wall time: 15.4 s


In [5]:
df_polygon

A0                            A1                   A2  \
0      44595357  2013189796191940668000000000  4825010100100120003   
1      44392386  1983189652331938159700000000  4825010100100160007   
2      44643939  1985189673551938250300000000  4825010100100160009   
3      44652157  1984189639821937683200000000  4825010100100160009   
4      44411698  1997189662551937232700000000  4825010100100240000   
...         ...                           ...                  ...   
53955  44604134  2006199657551945655000000000  4825037030102080026   
53956  44614473  2007199673951945433900000000  4825037030102080030   
53957  44614476  2007199666171945486300000000  4825037030102080030   
53958  44415948  2009199682541945349400000000  4825037030102080031   
53959  44597241  1993199694571946224000000000  4825037030102360001   

               A3                A4 A5  A6      A7         A8 A9  ...     A27  \
0      4825010100      경상남도 김해시 동상동  1  일반    12-3  102064839  1  ...    단독주택   
1      4825010100      경상남도 김해시 동상동  1  일반    16-7      17765  1  ...    단독주택   
2      4825010100      경상남도 김해시 동상동  1  일반    16-9      20572  1  ...    단독주택   
3      4825010100      경상남도 김해시 동상동  1  일반    16-9      44158  1  ...    단독주택   
4      4825010100      경상남도 김해시 동상동  1  일반      24      53973  1  ...    단독주택   
...           ...               ... ..  ..     ...        ... ..  ...     ...   
53955  4825037030  경상남도 김해시 대동면 조눌리  1  일반  208-26      14092  1  ...  노인복지시설   
53956  4825037030  경상남도 김해시 대동면 조눌리  1  일반  208-30      11710  1  ...    기타공장   
53957  4825037030  경상남도 김해시 대동면 조눌리  1  일반  208-30      11711  1  ...    기타공장   
53958  4825037030  경상남도 김해시 대동면 조눌리  1  일반  208-31  102026119  1  ...     소매점   
53959  4825037030  경상남도 김해시 대동면 조눌리  1  일반   236-1      32452  1  ...    마을회관   

      A28    A29   A30  A31  A32         A33         A34         A35  \
0       1    주거용  4.85  1.0  0.0  2012-11-14  2013-04-09  2022-01-17   
1       1    주거용  3.30  1.0  0.0        None  1983-12-31  2022-01-17   
2       1    주거용  0.00  1.0  0.0        None  1985-06-18  2022-01-17   
3       1    주거용  0.00  1.0  0.0        None  1984-05-08  2022-01-17   
4       1    주거용  3.90  1.0  0.0  1996-10-23  1997-06-12  2022-01-17   
...    ..    ...   ...  ...  ...         ...         ...         ...   
53955   6  문교사회용  4.75  1.0  0.0  2006-03-02  2006-09-11  2022-01-17   
53956   4    공업용  6.10  2.0  0.0  2007-03-22  2007-05-02  2022-01-17   
53957   4    공업용  6.10  1.0  0.0  2006-10-18  2007-03-06  2022-01-17   
53958   2    상업용  8.00  2.0  0.0  2007-06-12  2009-08-17  2022-01-17   
53959   2    상업용  0.00  2.0  0.0        None  1993-09-25  2022-01-17   

                                                geometry  
0      POLYGON ((1126333.712 1695375.093, 1126332.365...  
1      POLYGON ((1126184.269 1695118.352, 1126183.302...  
2      POLYGON ((1126210.129 1695131.023, 1126210.042...  
3      POLYGON ((1126183.150 1695073.289, 1126172.536...  
4      POLYGON ((1126195.806 1695031.773, 1126199.859...  
...                                                  ...  
53955  POLYGON ((1136179.752 1696023.231, 1136179.839...  
53956  POLYGON ((1136199.699 1696013.641, 1136206.680...  
53957  POLYGON ((1136198.852 1696014.151, 1136185.469...  
53958  POLYGON ((1136211.663 1695992.467, 1136204.782...  
53959  POLYGON ((1136215.906 1696080.672, 1136215.994...  

[53960 rows x 37 columns]

In [6]:
# 좌표계 변환
df_polygon = df_polygon.to_crs(epsg=4326)

In [7]:
import folium

In [8]:
# Geometry만 불러오기
# API json Polygon 대체
df_polygon = df_polygon.astype({'geometry':str})
geo_list =  df_polygon['geometry']

In [9]:
# 샘플 1000개 
geo_list = geo_list[:1000]

In [10]:
geo_list

0      POLYGON ((128.888574 35.245842, 128.888559 35....
1      POLYGON ((128.886893 35.243547, 128.886883 35....
2      POLYGON ((128.887179 35.243658, 128.887178 35....
3      POLYGON ((128.886873 35.24314, 128.886756 35.2...
4      POLYGON ((128.887006 35.242765, 128.887051 35....
                             ...                        
995    POLYGON ((128.883199 35.233822, 128.883094 35....
996    POLYGON ((128.882964 35.233736, 128.882955 35....
997    POLYGON ((128.882794 35.233703, 128.882787 35....
998    POLYGON ((128.883099 35.233565, 128.882911 35....
999    POLYGON ((128.883283 35.233599, 128.883121 35....
Name: geometry, Length: 1000, dtype: object

In [11]:
# json 형식처럼 불러오기 위한 가공 과정 
for n in tqdm(range(len(geo_list))):
    geo_list[n] = geo_list[n].split('((')[1]
for n in tqdm(range(len(geo_list))):
    geo_list[n] = geo_list[n].split('))')[0]
for n in tqdm(range(len(geo_list))):
    geo_list[n] = geo_list[n].split(', ')
for n in tqdm(range(len(geo_list))):
    for i in range(len(geo_list[n])):
        geo_list[n][i] = geo_list[n][i].split(' ')
for n in tqdm(range(len(geo_list))):
    for i in range(len(geo_list[n])):
        for h in range(len(geo_list[n][i])):
            geo_list[n][i][h] = float(geo_list[n][i][h])

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
geo_list

0      [[128.888574, 35.245842], [128.888559, 35.2458...
1      [[128.886893, 35.243547], [128.886883, 35.2435...
2      [[128.887179, 35.243658], [128.887178, 35.2436...
3      [[128.886873, 35.24314], [128.886756, 35.24311...
4      [[128.887006, 35.242765], [128.887051, 35.2427...
                             ...                        
995    [[128.883199, 35.233822], [128.883094, 35.2338...
996    [[128.882964, 35.233736], [128.882955, 35.2338...
997    [[128.882794, 35.233703], [128.882787, 35.2337...
998    [[128.883099, 35.233565], [128.882911, 35.2335...
999    [[128.883283, 35.233599], [128.883121, 35.2335...
Name: geometry, Length: 1000, dtype: object

In [13]:
# 경도, 위도 순으로 나타내기 위한 오름차순 정렬
for n in tqdm(range(len(geo_list))):
    for i in range(len(geo_list[n])):
        geo_list[n][i].sort()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
# 시작지점 중앙좌표
center = [35.236584, 128.884861]

# 맵 시작지점, 줌
m=folium.Map(location=center, zoom_start=16)

# 폴리곤 리스트
location_data = geo_list

# 맵 그리기
folium.Polygon(locations=location_data, fill=True, tooltip='Polygon').add_to(m)

# 맵 시각화
m

In [ ]:
# HTML로 저장

center = [35.236584, 128.884861]
m=folium.Map(location=center, zoom_start=16)

location_data = geo_list

folium.Polygon(locations=location_data, fill=True, tooltip='Polygon').add_to(m)

m.save('a.html')